In [53]:
import os
from pathlib import Path
import pandas as pd
import json
from epjson.epjson_idf import EpJsonIDF
from epjson.simulate_cluster import SimulateCluster

def run_single_building(
        config_dict, 
        output_dir, 
        weight_map,
        resource_dir=Path("tests/resources"), 
        idf_dir=Path("tests/resources/idfs/"),
        eplus_loc=Path("C:\EnergyPlusV22-2-0"), 
        update_epsjon=True
        ):
    
    weights_df = pd.read_json(resource_dir / "shoebox-weights.json")
    weights_df = weights_df[weights_df["building_id"] == config_dict["building_id"]]
    idfs = set(weights_df.ShoeboxPath.to_list())

    ### MUTATIONS ###

    # Get retrofit scenario template data
    scenario_dir = resource_dir / f"templates_RETRO_{config_dict['retrofit_scenario']}_SCHED_{config_dict['schedules_scenario']}_LAB_{config_dict['lab_scenario']}.csv"
    features_df = pd.read_csv(scenario_dir)
    print(scenario_dir)
    with open(resource_dir / "template_features_map.json", "r") as f:
        features_map = json.load(f)
    with open(resource_dir / "schedule_bunches.json", "r") as f:
        schedules = json.load(f)
        
    if update_epsjon:
        for i, idf in enumerate(idfs):
            try:
                idf_path = idf_dir / idf
                epjson = EpJsonIDF.from_epjson(idf_path.with_suffix(".epjson"), eplus_loc=eplus_loc)
                features_series = features_df[(features_df.TYPOLOGY == config_dict["TYPOLOGY"]) & (features_df.AGE_ERA == config_dict["AGE_ERA"])].reset_index(drop=True)
                features_series = features_series.iloc[0]
                # Set DHW flow rate based on sb area
                peak_dhw_pp = features_series["DHWFlowRate"]
                occupants = features_series["PeopleDensity"] * (weights_df.iloc[i]["core_area"])+(weights_df.iloc[i]["perim_area"])
                features_series["DHWFlowRate"] = peak_dhw_pp * occupants

                cols = set(features_series.index) & set(features_map.keys())

                epjson.replace_features(features_series[cols], features_map)
                epjson.update_schedules(schedules)
                epjson.add([(
                    "Output:SQLite",
                    {"Output:SQLite 1": {"option_type": "Simple"}}
                    )])
                epjson.epjson["ZoneInfiltration:DesignFlowRate"]["CoreInfiltration"].update(
                    {"air_changes_per_hour": 0}
                )
                epjson.epjson["WaterUse:Equipment"]["DHW Perimeter"].update(
                    {"peak_flow_rate": 0}
                )
                
                # new_path = replace_shoebox_path(idf_path).with_suffix(".epJSON")
                new_path = output_dir / Path(idf).with_suffix(".epJSON")
                epjson.save_as(new_path)
                epjson.save_idf(output_path=new_path.parent) #suffix="_test",

            except Exception as e:
                print(f"FAILED ON {idf}")
                raise e
    
    ### SIMULATION ###
    # Get new paths for updated shoeboxes
    # new_idfs = weights_df["ShoeboxPath"].apply(replace_shoebox_path)
    new_idfs = weights_df["ShoeboxPath"].apply(lambda x: output_dir / Path(x))
    weights_df["ShoeboxPath"] = new_idfs
    
    print("BUILDING CLUSTER")
    idf_cluster = SimulateCluster(
        idf_list = set(new_idfs),
        epw = config_dict["epw"],
        weights_df = weights_df,
        building_col = "building_id",
        eplus_location=eplus_loc,
        weight_map = weight_map
    )
    print("SIMULATING CLUSTER")
    errors = idf_cluster.parallel_simulate() # Fix errors dict

    print("PROCESSING CLUSTER RESULTS")
    ### POST-PROCESS ###
    r = idf_cluster.fetch_building_results_parallel([config_dict["building_id"]])

    return r, errors, idf_cluster

In [ ]:
config_dict = {
    "building_id": "62",
    "epw": "D:\\Users\\zoelh\\Dropbox (MIT)\\4.S42 Campus Decarb\\Energy Modeling\\epws\\BaselineFile.epw",
    "retrofit_scenario": "baseline",
    "schedules_scenario": "baseline",
    "lab_scenario": "baseline",
    "TYPOLOGY": "Residential",
    "AGE_ERA": 1945,
    "lab_weight": 0.268159426
}

sb_weight_map = {
    "Perimeter": "PerimeterAreaWeight",
    "Core": "CoreAreaWeight"
}

office_res, office_err, office_cluster = run_single_building(config_dict, 
                               weight_map = sb_weight_map,
                               output_dir = Path(os.getcwd()) / "eplus", 
                               idf_dir=Path("D:\\Users\\zoelh\\Dropbox (MIT)\\4.S42 Campus Decarb\\Energy Modeling\\idfs\\gis\\simple_shoeboxes"),
                               update_epsjon=True
                               )

In [55]:
office_res[config_dict["building_id"]].sum()

C:\Users\zoelh\AppData\Local\Temp\ipykernel_575388\1685267888.py:1: FutureWarning: The default value of numeric_only in DataFrame.sum is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  office_res[config_dict["building_id"]].sum()


Zone Lights Electricity Energy                                                               197316.252678
Electric Equipment Electricity Energy                                                        403749.280475
Zone Ideal Loads Supply Air Total Heating Energy                                             870017.083634
Zone Ideal Loads Supply Air Total Cooling Energy                                              471255.30736
Water Use Equipment Heating Energy                                                            413424.51759
Zone Lights Electricity Energy_norm                                                               15.11091
Electric Equipment Electricity Energy_norm                                                       30.920002
Zone Ideal Loads Supply Air Total Heating Energy_norm                                            66.627809
Zone Ideal Loads Supply Air Total Cooling Energy_norm                                            36.089761
Water Use Equipment Heating Energy_no

Run all archetypes

In [ ]:
sbs = pd.read_json("tests/resources/shoebox-weights.json")
sbs = sbs.groupby(["TYPOLOGY", "AGE_ERA"]).first().reset_index()
res = {}
for i, row in sbs.iterrows():
    config_dict = {
        "building_id": row.building_id,
        "epw": "D:\\Users\\zoelh\\Dropbox (MIT)\\4.S42 Campus Decarb\\Energy Modeling\\epws\\USA_MA_Boston-Logan.Intl.AP.725090_TMY3.epw",
        # "epw": "D:\\Users\\zoelh\\Dropbox (MIT)\\4.S42 Campus Decarb\\Energy Modeling\\epws\\BaselineFile.epw",
        "retrofit_scenario": "baseline",
        "schedules_scenario": "baseline",
        "lab_scenario": "baseline",
        "TYPOLOGY": row.TYPOLOGY,
        "AGE_ERA": row.AGE_ERA,
        "lab_weight": 0.8
    }
    office_res, office_err, office_cluster = run_single_building(config_dict, 
                                weight_map = sb_weight_map,
                                output_dir = Path(os.getcwd()) / "eplus", 
                                idf_dir=Path("D:\\Users\\zoelh\\Dropbox (MIT)\\4.S42 Campus Decarb\\Energy Modeling\\idfs\\gis\\simple_shoeboxes"),
                                update_epsjon=True
                                )
    print(row.TYPOLOGY, row.AGE_ERA)
    res[(row.TYPOLOGY, row.AGE_ERA, row.building_id)] = office_res[row.building_id].sum()
    

In [57]:
norm_cols = ['Zone Lights Electricity Energy_norm',
       'Electric Equipment Electricity Energy_norm',
       'Zone Ideal Loads Supply Air Total Heating Energy_norm',
       'Zone Ideal Loads Supply Air Total Cooling Energy_norm',
       'Water Use Equipment Heating Energy_norm']
pd.DataFrame.from_dict(res).T[norm_cols]

Zone Lights Electricity Energy_norm  \
Athletic             1945 51                            174.709437   
                     1980 W31                           152.870765   
Lab and Mixed Use    1945 1                             104.825664   
                     1980 13                            104.825662   
                     2015 12                            104.825661   
Office and Mixed Use 1945 50                            104.825659   
                     1980 W11                            50.665741   
Residential          1945 62                              15.11091   
                     1980 NW30                           15.110911   
                     2015 E37                             15.11091   
Support Dominant     1945 42                             26.570394   
                     1980 E23                            26.570394   
                     2015 42C                            26.570393   

                               Electric Equipment Electricity Energy_norm  \
Athletic             1945 51                                   436.773593   
                     1980 W31                                  436.773613   
Lab and Mixed Use    1945 1                                     480.45096   
                     1980 13                                   349.418875   
                     2015 12                                   105.553617   
Office and Mixed Use 1945 50                                   480.450939   
                     1980 W11                                  349.418901   
Residential          1945 62                                    30.920002   
                     1980 NW30                                  30.920005   
                     2015 E37                                   30.920003   
Support Dominant     1945 42                                   132.851972   
                     1980 E23                                  132.851969   
                     2015 42C                                  132.851964   

                               Zone Ideal Loads Supply Air Total Heating Energy_norm  \
Athletic             1945 51                                            11.736536      
                     1980 W31                                            0.671785      
Lab and Mixed Use    1945 1                                             15.917859      
                     1980 13                                             8.999106      
                     2015 12                                             13.82172      
Office and Mixed Use 1945 50                                            10.387457      
                     1980 W11                                           13.012336      
Residential          1945 62                                            88.650217      
                     1980 NW30                                          57.310861      
                     2015 E37                                           47.862422      
Support Dominant     1945 42                                            51.167153      
                     1980 E23                                           47.714292      
                     2015 42C                                           28.957659      

                               Zone Ideal Loads Supply Air Total Cooling Energy_norm  \
Athletic             1945 51                                            76.079774      
                     1980 W31                                           68.838449      
Lab and Mixed Use    1945 1                                            549.302325      
                     1980 13                                           444.702001      
                     2015 12                                           237.188842      
Office and Mixed Use 1945 50                                           558.040611      
                     1980 W11                                          370.145256      
Residential          1945 62      

In [ ]:
sbs = pd.read_json("tests/resources/shoebox-weights.json")
sbs = sbs.groupby(["TYPOLOGY", "AGE_ERA"]).first().reset_index()
partialres = {}
for i, row in sbs.iterrows():
    config_dict = {
        "building_id": row.building_id,
        "epw": "D:\\Users\\zoelh\\Dropbox (MIT)\\4.S42 Campus Decarb\\Energy Modeling\\epws\\USA_MA_Boston-Logan.Intl.AP.725090_TMY3.epw",
        "retrofit_scenario": "partial",
        "schedules_scenario": "baseline",
        "lab_scenario": "baseline",
        "TYPOLOGY": row.TYPOLOGY,
        "AGE_ERA": row.AGE_ERA,
        "lab_weight": 0.8
    }
    office_res, office_err, office_cluster = run_single_building(config_dict, 
                                weight_map = sb_weight_map,
                                output_dir = Path(os.getcwd()) / "eplus", 
                                idf_dir=Path("D:\\Users\\zoelh\\Dropbox (MIT)\\4.S42 Campus Decarb\\Energy Modeling\\idfs\\gis\\simple_shoeboxes"),
                                update_epsjon=True
                                )
    print(row.TYPOLOGY, row.AGE_ERA)
    partialres[(row.TYPOLOGY, row.AGE_ERA, row.building_id)] = office_res[row.building_id].sum()
    

In [60]:
pd.DataFrame.from_dict(partialres).T[norm_cols]

Zone Lights Electricity Energy_norm  \
Athletic             1945 51                             87.354719   
                     1980 W31                             78.61925   
Lab and Mixed Use    1945 1                              52.412832   
                     1980 13                             52.412831   
                     2015 12                              52.41283   
Office and Mixed Use 1945 50                              52.41283   
                     1980 W11                             25.33287   
Residential          1945 62                              7.555455   
                     1980 NW30                            7.555456   
                     2015 E37                             7.555455   
Support Dominant     1945 42                             13.285197   
                     1980 E23                            13.285197   
                     2015 42C                            13.285196   

                               Electric Equipment Electricity Energy_norm  \
Athletic             1945 51                                   436.773593   
                     1980 W31                                  436.773613   
Lab and Mixed Use    1945 1                                     480.45096   
                     1980 13                                   349.418875   
                     2015 12                                   105.553617   
Office and Mixed Use 1945 50                                   480.450939   
                     1980 W11                                  349.418901   
Residential          1945 62                                    30.920002   
                     1980 NW30                                  30.920005   
                     2015 E37                                   30.920003   
Support Dominant     1945 42                                   132.851972   
                     1980 E23                                  132.851969   
                     2015 42C                                  132.851964   

                               Zone Ideal Loads Supply Air Total Heating Energy_norm  \
Athletic             1945 51                                            14.052881      
                     1980 W31                                            0.034153      
Lab and Mixed Use    1945 1                                              11.00495      
                     1980 13                                             4.965824      
                     2015 12                                            11.053994      
Office and Mixed Use 1945 50                                             7.229004      
                     1980 W11                                            4.757419      
Residential          1945 62                                            82.104237      
                     1980 NW30                                           49.01215      
                     2015 E37                                           41.198976      
Support Dominant     1945 42                                            18.739557      
                     1980 E23                                           21.914194      
                     2015 42C                                           10.719523      

                               Zone Ideal Loads Supply Air Total Cooling Energy_norm  \
Athletic             1945 51                                             65.84975      
                     1980 W31                                           60.661892      
Lab and Mixed Use    1945 1                                             91.141196      
                     1980 13                                            63.203425      
                     2015 12                                            43.308154      
Office and Mixed Use 1945 50                                            59.578521      
                     1980 W11                                           69.199485      
Residential          1945 62      

In [ ]:
sbs = pd.read_json("tests/resources/shoebox-weights.json")
sbs = sbs.groupby(["TYPOLOGY", "AGE_ERA"]).first().reset_index()
fullres = {}
for i, row in sbs.iterrows():
    config_dict = {
        "building_id": row.building_id,
        "epw": "D:\\Users\\zoelh\\Dropbox (MIT)\\4.S42 Campus Decarb\\Energy Modeling\\epws\\USA_MA_Boston-Logan.Intl.AP.725090_TMY3.epw",
        "retrofit_scenario": "full",
        "schedules_scenario": "baseline",
        "lab_scenario": "baseline",
        "TYPOLOGY": row.TYPOLOGY,
        "AGE_ERA": row.AGE_ERA,
        "lab_weight": 0.8
    }
    office_res, office_err, office_cluster = run_single_building(config_dict, 
                                weight_map = sb_weight_map,
                                output_dir = Path(os.getcwd()) / "eplus", 
                                idf_dir=Path("D:\\Users\\zoelh\\Dropbox (MIT)\\4.S42 Campus Decarb\\Energy Modeling\\idfs\\gis\\simple_shoeboxes"),
                                update_epsjon=True
                                )
    print(row.TYPOLOGY, row.AGE_ERA)
    fullres[(row.TYPOLOGY, row.AGE_ERA, row.building_id)] = office_res[row.building_id].sum()
    

In [62]:
pd.DataFrame.from_dict(fullres).T[norm_cols]

Zone Lights Electricity Energy_norm  \
Athletic             1945 51                             87.354719   
                     1980 W31                             78.61925   
Lab and Mixed Use    1945 1                              52.412832   
                     1980 13                             52.412831   
                     2015 12                              52.41283   
Office and Mixed Use 1945 50                              52.41283   
                     1980 W11                             25.33287   
Residential          1945 62                              7.555455   
                     1980 NW30                            7.555456   
                     2015 E37                             7.555455   
Support Dominant     1945 42                             13.285197   
                     1980 E23                            13.285197   
                     2015 42C                            13.285196   

                               Electric Equipment Electricity Energy_norm  \
Athletic             1945 51                                   436.773593   
                     1980 W31                                  436.773613   
Lab and Mixed Use    1945 1                                     480.45096   
                     1980 13                                   349.418875   
                     2015 12                                   105.553617   
Office and Mixed Use 1945 50                                   480.450939   
                     1980 W11                                  349.418901   
Residential          1945 62                                    30.920002   
                     1980 NW30                                  30.920005   
                     2015 E37                                   30.920003   
Support Dominant     1945 42                                   132.851972   
                     1980 E23                                  132.851969   
                     2015 42C                                  132.851964   

                               Zone Ideal Loads Supply Air Total Heating Energy_norm  \
Athletic             1945 51                                                  0.0      
                     1980 W31                                                 0.0      
Lab and Mixed Use    1945 1                                                   0.0      
                     1980 13                                             0.002462      
                     2015 12                                             1.587394      
Office and Mixed Use 1945 50                                                  0.0      
                     1980 W11                                            0.040111      
Residential          1945 62                                            30.460753      
                     1980 NW30                                           12.12782      
                     2015 E37                                           11.881545      
Support Dominant     1945 42                                             6.900067      
                     1980 E23                                            6.243716      
                     2015 42C                                             4.43932      

                               Zone Ideal Loads Supply Air Total Cooling Energy_norm  \
Athletic             1945 51                                            61.736831      
                     1980 W31                                           59.307497      
Lab and Mixed Use    1945 1                                              88.42994      
                     1980 13                                            61.598259      
                     2015 12                                            41.221423      
Office and Mixed Use 1945 50                                            58.401299      
                     1980 W11                                           66.612453      
Residential          1945 62      

In [ ]:
sbs = pd.read_json("tests/resources/shoebox-weights.json")
sbs = sbs.groupby(["TYPOLOGY", "AGE_ERA"]).first().reset_index()
allfullres = {}
for i, row in sbs.iterrows():
    config_dict = {
        "building_id": row.building_id,
        "epw": "D:\\Users\\zoelh\\Dropbox (MIT)\\4.S42 Campus Decarb\\Energy Modeling\\epws\\USA_MA_Boston-Logan.Intl.AP.725090_TMY3.epw",
        "retrofit_scenario": "full",
        "schedules_scenario": "full",
        "lab_scenario": "baseline",
        "TYPOLOGY": row.TYPOLOGY,
        "AGE_ERA": row.AGE_ERA,
        "lab_weight": 0.8
    }
    office_res, office_err, office_cluster = run_single_building(config_dict, 
                                weight_map = sb_weight_map,
                                output_dir = Path(os.getcwd()) / "eplus", 
                                idf_dir=Path("D:\\Users\\zoelh\\Dropbox (MIT)\\4.S42 Campus Decarb\\Energy Modeling\\idfs\\gis\\simple_shoeboxes"),
                                update_epsjon=True
                                )
    print(row.TYPOLOGY, row.AGE_ERA)
    allfullres[(row.TYPOLOGY, row.AGE_ERA, row.building_id)] = office_res[row.building_id].sum()


In [64]:
pd.DataFrame.from_dict(allfullres).T[norm_cols]

Zone Lights Electricity Energy_norm  \
Athletic             1945 51                             87.354719   
                     1980 W31                             78.61925   
Lab and Mixed Use    1945 1                              32.648827   
                     1980 13                             32.648826   
                     2015 12                             32.648826   
Office and Mixed Use 1945 50                             32.648825   
                     1980 W11                            32.648829   
Residential          1945 62                              7.555455   
                     1980 NW30                            7.555456   
                     2015 E37                             7.555455   
Support Dominant     1945 42                             27.207356   
                     1980 E23                            27.207355   
                     2015 42C                            27.207354   

                               Electric Equipment Electricity Energy_norm  \
Athletic             1945 51                                   436.773593   
                     1980 W31                                  436.773613   
Lab and Mixed Use    1945 1                                     480.45096   
                     1980 13                                   349.418875   
                     2015 12                                   105.553617   
Office and Mixed Use 1945 50                                   480.450939   
                     1980 W11                                  349.418901   
Residential          1945 62                                    30.920002   
                     1980 NW30                                  30.920005   
                     2015 E37                                   30.920003   
Support Dominant     1945 42                                   132.851972   
                     1980 E23                                  132.851969   
                     2015 42C                                  132.851964   

                               Zone Ideal Loads Supply Air Total Heating Energy_norm  \
Athletic             1945 51                                                  0.0      
                     1980 W31                                                 0.0      
Lab and Mixed Use    1945 1                                                   0.0      
                     1980 13                                                  0.0      
                     2015 12                                             0.100986      
Office and Mixed Use 1945 50                                                  0.0      
                     1980 W11                                                 0.0      
Residential          1945 62                                            17.905335      
                     1980 NW30                                           5.145644      
                     2015 E37                                            4.954598      
Support Dominant     1945 42                                             0.386677      
                     1980 E23                                            0.336192      
                     2015 42C                                            0.211382      

                               Zone Ideal Loads Supply Air Total Cooling Energy_norm  \
Athletic             1945 51                                            61.677695      
                     1980 W31                                           59.276548      
Lab and Mixed Use    1945 1                                             71.013608      
                     1980 13                                            47.978383      
                     2015 12                                            33.496247      
Office and Mixed Use 1945 50                                            44.876122      
                     1980 W11                                           55.365314      
Residential          1945 62      